# Compare models with SST

This notebook shows an example of how to;
* load SST data using the `default` driver
* load NEMO model data using the `default` driver
* interpolate the model data to the SST location using `xarray`
* calculate metrics
* visualise the results using `cartopy`

In [ ]:
import sys

sys.path.append("../")
import xarray as xr
import numpy as np
import zapata.data as zdat

import warnings

warnings.filterwarnings("ignore")


In [ ]:
# check catalogue contents
zdat.inquire_catalogue()

modelcat = 'BSFS-NRT_daily'

zdat.inquire_catalogue(dataset=modelcat, info=True)


In [ ]:
## Read Model data
model = zdat.read_data(dataset=modelcat, var='votemper', period=[2018, 2018], level=[3.])

if not np.issubdtype(model['time'].dtype, np.datetime64):
    try:
        from xarray.coding.times import cftime_to_nptime

        model['time'] = cftime_to_nptime(model['time'])
    except ValueError as e:
        print(e)
        pass


In [ ]:
lat = model.nav_lat[:, 0]
lon = model.nav_lon[0, :]
model = model.assign_coords({"lon": lon, "lat": lat}).drop('lev')
model = model.squeeze()


In [ ]:
sstcat = 'SST'
zdat.inquire_catalogue(dataset=sstcat, info=True)

## Read SST data
sst = zdat.read_data(dataset=sstcat, var='sea_surface_temperature', period=[2018, 2018])


In [ ]:
#Filter SST data
sst_qc = zdat.read_data(dataset=sstcat, var='quality_level', period=[2018, 2018])
sst_filtered = sst.where(sst_qc == 4)

In [ ]:
# Convert from Kelvin to Celsius
if sst_filtered.attrs['units'] == 'kelvin':
    sst_filtered -= 273.15
    sst_filtered.attrs['units'] = 'degrees_C'

if not np.issubdtype(sst_filtered['time'].dtype, np.datetime64):
    try:
        from xarray.coding.times import cftime_to_nptime

        sst_filtered['time'] = cftime_to_nptime(sst_filtered['time'])
    except ValueError as e:
        print(e)
        pass


In [ ]:
#Interpolate the model data to the SST location
model.chunk({'time': model['time'].size})
model = model.interp(sst_filtered.coords, method='nearest').load()


In [ ]:
result = xr.Dataset()

model.data[model.data == 0] = np.nan
bias = model - sst_filtered

result['bias'] = bias.mean(dim='time')
result['rmse'] = xr.ufuncs.sqrt((bias ** 2.).mean(dim='time'))

result['bias_ts'] = bias.mean(dim=['lat', 'lon'])
result['rmse_ts'] = xr.ufuncs.sqrt((bias ** 2.)).mean(dim=['lat', 'lon'])

result.to_dataframe()


In [ ]:
import matplotlib
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

%matplotlib inline
import matplotlib.pyplot as plt

for metric in ['Bias', 'RMSE']:
    fig = plt.figure(dpi=120)
    ax = plt.axes()

    plt.plot(result['time'].data, result['%s_ts' % metric.lower()], '.-')

    plt.title('sst')
    plt.ylabel('%s [%s]' % (metric, model.attrs['units']))
    plt.xlabel('Time')
    plt.grid(True, c='silver', lw=1, ls=':')
    plt.tight_layout()
    fig.autofmt_xdate()

    plt.savefig('sst_%s.png' % (metric.lower()))

    plt.show()


In [ ]:
import zapata.mapping as zmap

bigtit = 'SST'
left1 = 'Bias'
left2 = 'RMSE'

fig, ax, pro = zmap.init_figure(2, 1, 'Pacific', constrained_layout=False, figsize=(24, 12))

handle = zmap.xmap(result['bias'], [], pro, ax=ax[0], contour=False, xlimit=[27.32, 41.96], ylimit=[40.86, 46.80], \
                   maintitle=bigtit, lefttitle=left1, cmap='coolwarm')
# Add horizontal colorbar
zmap.add_colorbar(fig, handle['filled'], ax[0], colorbar_size=0.01, label_size=10, edges=True)

han1 = zmap.xmap(result['rmse'], [], pro, ax=ax[1], contour=False, xlimit=[27.32, 41.96], ylimit=[40.86, 46.80], \
                 maintitle=bigtit, lefttitle=left2, cmap='coolwarm')
zmap.add_colorbar(fig, han1['filled'], ax[1], colorbar_size=0.01, label_size=10, edges=True)

fig.subplots_adjust(wspace=0, hspace=0.2)

plt.savefig('SST.pdf')
plt.show()
